In [ ]:
using Distributed
using Plots

# Problem 1.1A & B

In [ ]:
"""
    wave_equation(psi, c)

Compute the second time derivative of the wave function using the 1D wave equation.

Uses the finite difference approximation of the wave equation: ∂²ψ/∂t² = c² ∂²ψ/∂x²

# Arguments
- `psi`: Vector of wave function values at discrete spatial points
- `c`: Wave speed (propagation velocity)

# Returns
- `d2psi_dt2`: Vector of second time derivatives at interior points (length = length(psi) - 2)

# Notes
- Uses central difference approximation for the second spatial derivative
- Assumes fixed boundary conditions (boundary points are excluded from computation)
- Requires `L` (domain length) and `N` (number of points) to be defined in the current scope
- The spatial step size dx = L/N is used in the finite difference formula
"""
function wave_equation(psi, c)
    # Starts from second point to second last point
    d2psi_dx2 = c^2 * (psi[1:end-2] - 2*psi[2:end-1] + psi[3:end]) / (L/N)^2
    d2psi_dt2 = d2psi_dx2
    return d2psi_dt2
end


In [ ]:
initial_condition_0 = (x) -> sin( x)
initial_condition_1 = (x) -> sin(2pi * x)
initial_condition_2 = (x) -> sin(5pi * x)
initial_condition_3 = (x) -> (x > 1/5 && x < 2/5) ? sin(5pi * x) : 0.0


In [ ]:
# Test finite difference approximation of the wave equation
L = 2pi
N = 100
dx = L/N
x = 0:dx:L
psi = initial_condition_0.(x)
dpsi_dt = zeros(N)
c = 1
d2psi_dt2 = wave_equation(psi, c)  
# Plot initial condition and second time derivative
plot(x, psi, label="Initial Condition", title="Wave Equation Test")
plot!(x[2:end-1], d2psi_dt2, label="Second Time Derivative", xlabel="x", ylabel="Value")

In [ ]:
function propagate_psi(psi_0_f::Function; L=1, N=100, c=1, t_0=0, t_f=1, dt=0.01)
    # Initial condition at t=0
    psi_x_i = psi_0_f.(range(0, L, length=N))

    # The string is at rest at t=0
    # Array is two elements shorter due to second derivative calculation
    dpsi_dt_i = zeros(N-2)

    # Boundaries are fixed to zero
    psi_x_i[1] = 0
    psi_x_i[end] = 0
    
    # Initialize the array to store the results
    psi_x_t = zeros(N, Int((t_f - t_0) / dt))
    psi_x_t[:, 1] = psi_x_i  

    # Time propagation using finite difference method
    for n in 1:size(psi_x_t, 2)-1
        d2psi_dt2 = wave_equation(psi_x_t[:, n], c)
        # Use Euler's method to update psi and its time derivative
        dpsi_dt_i += d2psi_dt2 * dt
        psi_x_t[2:end-1, n+1] = psi_x_t[2:end-1, n] + dpsi_dt_i * dt
    end

    return psi_x_t
end

In [ ]:
t_0 = 0
t_f = 1
dt = 0.001
c=1
L=1
N=1000

In [ ]:
solution_1 = propagate_psi(initial_condition_1, L=L, N=N, c=c, t_0=t_0, t_f=t_f, dt=dt)
solution_2 = propagate_psi(initial_condition_2, L=L, N=N, c=c, t_0=t_0, t_f=t_f, dt=dt)
solution_3 = propagate_psi(initial_condition_3, L=L, N=N, c=c, t_0=t_0, t_f=t_f, dt=dt)

In [ ]:
# Preperare distributed
addprocs(Sys.CPU_THREADS - nworkers())
print("Number of workers: ", nworkers(), "\n")
@everywhere using Plots

In [ ]:
# Animate the solution and save frames
default(legend=false)
i_total = Int((t_f - t_0) / dt)
filenames = [lpad(i, 6, "0") * ".png" for i in 1:i_total]
@sync @distributed for i in range(1, stop=i_total, length=i_total)
    i_int = Int(i)

    x = range(0, L, length=N)

    # create a plot with 3 subplots and a custom layout
    p = plot(x, solution_1[:, i_int], ylim=(-1, 1), title="Time: $(round(i_int*dt, digits=2)) s", xlabel="Position along string", ylabel="Displacement")
    plot!(x, solution_2[:, i_int])
    plot!(x, solution_3[:, i_int])
    savefig("tmp/$(lpad(i_int, 6, "0")).png")
end

In [ ]:
# Create an animation from the saved frames
anim = Animation("tmp", filenames)
Plots.buildanimation(anim, "tmp/anim.gif", fps=30, show_msg=false)